__1. Улучшение качества  POS-taggera.__

In [1]:
import pyconll

In [2]:
import os
import urllib.request

path_dir = "./datasets"
path_train = f"{path_dir}/ru_syntagrus-ud-train.conllu"
path_dev = f"{path_dir}/ru_syntagrus-ud-dev.conllu"

os.mkdir(path_dir) if not os.path.exists(path_dir) else print(f"{path_dir} exist")
    
url_train = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu'
url_dev = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu'

urllib.request.urlretrieve(url_train, path_train) if not os.path.exists(path_train) else print(f"{path_train} exist")
    
urllib.request.urlretrieve(url_dev, path_dev) if not os.path.exists(path_dev) else print(f"{path_dev} exist")

./datasets exist
./datasets/ru_syntagrus-ud-train.conllu exist
./datasets/ru_syntagrus-ud-dev.conllu exist


In [3]:
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

In [4]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [5]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [8]:
test_enc_labels = le.transform(test_label)

In [9]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=200)

In [10]:
X_train = hvectorizer.fit_transform(train_tok)

In [11]:
X_test = hvectorizer.transform(test_tok)

In [12]:
lr = LogisticRegression(penalty="elasticnet",
                        random_state=0, 
                        max_iter=30, 
                        solver="saga",
                        n_jobs=-1,
                        l1_ratio=0.1)

lr.fit(X_train, train_enc_labels);

In [13]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.7925976476931891

__1.1. Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000).__

In [14]:
import pandas as pd
import numpy as np
import nltk
import spacy
from spacy import displacy
import en_core_web_sm

In [15]:
combine_df = pd.read_pickle("data/processed_tweets.pkl")
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[when, father, is, dysfunct, and, is, so, self...","[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, for, lyft, credit, can, not, use, caus...","[thanks, for, lyft, credit, can, not, use, cau..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, your, majesti]","[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, with, yoyou, all, t..."
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, now, motiv]","[factsguide, society, now, motivation]"


In [16]:
nlp = en_core_web_sm.load()

In [17]:
max_word_limit = 1000000

clean_tweets = ''.join(combine_df['clean_tweet'])[:max_word_limit]
article = nlp(clean_tweets)
# displacy.render(article, jupyter=True, style='ent')

In [18]:
from collections import Counter

In [19]:
n_top = 20

Counter([token.label_ for token in article.ents]).most_common(n_top)

[('DATE', 2453),
 ('PERSON', 2200),
 ('GPE', 1286),
 ('ORG', 1049),
 ('TIME', 482),
 ('NORP', 467),
 ('CARDINAL', 279),
 ('ORDINAL', 171),
 ('FAC', 122),
 ('LOC', 92),
 ('EVENT', 49),
 ('PRODUCT', 43),
 ('LANGUAGE', 15),
 ('WORK_OF_ART', 13),
 ('QUANTITY', 6),
 ('LAW', 2),
 ('MONEY', 1)]

__1.2. С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?__

In [20]:
label = 'PERSON'

Counter(token.text for token in article.ents if token.label_==label).most_common(n_top)

[('hea', 33),
 ('hu', 23),
 ('blur sun', 13),
 ('hillary', 10),
 ('christina grimmie', 10),
 ('bihday', 9),
 ('karen iqbal', 7),
 ('blog', 6),
 ('punjab', 6),
 ('dayam', 5),
 ('michelle', 5),
 ('possibility anne hathaway', 5),
 ('youam', 5),
 ('anton yelchin', 5),
 ('mm', 4),
 ('luis suarez', 4),
 ('don', 4),
 ('carl paladino', 4),
 ('jo cox', 4),
 ('spos', 3)]

In [21]:
label = 'ORG'

Counter(token.text for token in article.ents if token.label_==label).most_common(n_top)

[('app', 26),
 ('gop', 15),
 ('islam', 15),
 ('usd', 11),
 ('bong bing', 10),
 ('nba', 9),
 ('fed', 9),
 ('social analytics', 8),
 ('usa', 7),
 ('house', 7),
 ('congress', 6),
 ('sma', 6),
 ('euro', 6),
 ('eu', 6),
 ('mac', 6),
 ('bogota colombia', 6),
 ('amazon', 4),
 ('cnn', 4),
 ('un', 4),
 ('fbi', 3)]

__1.3. Повторим шаги из заданий 1.1. и 1.2., используя библиотеку nltk.__

In [22]:
# nltk.download()

In [23]:
any(word.istitle() for word in clean_tweets.split(' '))

False

In [24]:
tree = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(clean_tweets)))

In [25]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in tree if hasattr(chunk, 'label')}

set()

In [26]:
Counter(itm[1] for itm in tree).most_common(n_top)

[('NN', 41736),
 ('JJ', 19319),
 ('IN', 14484),
 ('NNS', 9177),
 ('RB', 8592),
 ('VB', 8287),
 ('VBP', 7915),
 ('DT', 7181),
 ('PRP', 7151),
 ('VBZ', 5464),
 ('VBG', 4683),
 ('TO', 4373),
 ('VBD', 3213),
 ('PRP$', 3172),
 ('CC', 2832),
 ('VBN', 2576),
 ('MD', 1915),
 ('WRB', 896),
 ('WP', 849),
 ('RP', 828)]

Так как nltk чувствителен к регистру, а все твиты в нижнем регистре, то nltk не может выполнить NER обработанного текста. В подобных случаях можно применять регулярные выражения, задавая для каждого класса свой паттерн.

__2.1. Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.__

__2.2. С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?__

__3. Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.__

Так как в связи с особенностью данных сравнить библиотеки не представляется возможным, то можно только предположить, что spacy, в данном случае, лучше, так как является более гибкой и быстрой. Nltk же работает дольше, но должен обладать более высокой точностью, так как использует классификацию  на основе деревьев.